### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [3]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 60
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [0]:
def batch_generator(df, batchsize=32):   
    
    while True:       
                
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0], batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class        
        
        yield batch_images, batch_labels          

In [12]:
a = batch_generator(train_df)
a

<generator object batch_generator at 0x7f771f5b2410>

### Build CNN Model

In [7]:
#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add Layers
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [0]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize)

In [10]:
model.fit_generator(train_generator, 
                          epochs=50,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
45/45 [==============================] - 15s 337ms/step - loss: 2.0795 - acc: 0.3510 - val_loss: 1.5817 - val_acc: 0.2983
Epoch 2/50
45/45 [==============================] - 13s 279ms/step - loss: 1.2657 - acc: 0.4830 - val_loss: 1.2278 - val_acc: 0.4787
Epoch 3/50
45/45 [==============================] - 12s 270ms/step - loss: 1.0255 - acc: 0.6049 - val_loss: 1.0727 - val_acc: 0.5497
Epoch 4/50
45/45 [==============================] - 12s 273ms/step - loss: 0.7801 - acc: 0.7247 - val_loss: 1.0058 - val_acc: 0.6037
Epoch 5/50
45/45 [==============================] - 12s 270ms/step - loss: 0.5671 - acc: 0.8142 - val_loss: 1.1817 - val_acc: 0.5568
Epoch 6/50
45/45 [==============================] - 12s 276ms/step - loss: 0.4142 - acc: 0.8615 - val_loss: 1.0336 - val_acc: 0.6648
Epoch 7/50
45/45 [==============================] - 12s 276ms/step - loss: 0.2776 - acc: 0.9215 - val_loss: 1.1968 - val_acc: 0.6108
Epoch 8/50
45/45 [===